## two cells below i used for for whisker, i think

In [ ]:
import pynwb
import numpy as np

nwb_path = r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb"

with pynwb.NWBHDF5IO(nwb_path, "r") as io:
    nwb = io.read()
    
    # Raw ephys - this is the broadband data you can feed to KS4
    ephys = nwb.acquisition["extracellular array recording"]
    print(f"Shape: {ephys.data.shape}")       # (65033216, 56) = samples x channels
    print(f"Rate: {ephys.rate}")               # ~30kHz
    print(f"Starting time: {ephys.starting_time}")  # offset into video timebase
    
    # Already spike-sorted units
    print(f"\nUnits: {len(nwb.units)}")
    print(nwb.units.colnames)
    
    # To extract raw data for KS4 (careful - large!):
    raw = ephys.data[:]  # loads full array into RAM
    np.save("raw_ephys.npy", raw)

Shape: (65033216, 56)
Rate: 29999.989041455377
Starting time: 399.0283126054678

Units: 14
('inhibitory', 'depth', 'layer', 'neuron_channel', 'spike_times', 'obs_intervals')


In [ ]:
from dandi.dandiapi import DandiAPIClient
from pathlib import Path

client = DandiAPIClient()
dandiset = client.get_dandiset("000231", "0.220904.1554")

for asset in dandiset.get_assets():
    if "KF134" in asset.path and "20180211T154918" in asset.path and "mkv" in asset.path:
        print(f"{asset.size/1e9:.2f} GB  {asset.path}")
        asset.download(Path(r"data\whisker_video_ephys.mkv"))
        break

6.60 GB  sub-KF134/sub-KF134_ses-20180211T154918_behavior+ecephys+image/26871407-4821-4cd0-bde7-38e14b3c13ba_external_file_0.mkv


In [1]:
from dandi.dandiapi import DandiAPIClient
from pynwb import NWBHDF5IO

with DandiAPIClient() as client:
    asset = client.get_dandiset("000231").get_asset_by_path(
        "sub-KF134/sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb"
    )
    url = asset.get_content_url(follow_redirects=1, strip_query=True)

io = NWBHDF5IO(url, mode="r", load_namespaces=True, driver="ros3")
nwb = io.read()

# What's inside?
print("Acquisition:", list(nwb.acquisition.keys()))
print("Processing:", list(nwb.processing.keys()))
print("Units:", nwb.units if nwb.units else "None")
print("Trials:", nwb.trials.to_dataframe().head() if nwb.trials else "None")
print("Electrodes:", nwb.electrodes.to_dataframe().head())

Acquisition: ['behavioral video', 'extracellular array recording']
Processing: ['behavior']
Units: units pynwb.misc.Units at 0x2514637048192
Fields:
  colnames: ['inhibitory' 'depth' 'layer' 'neuron_channel' 'spike_times'
 'obs_intervals']
  columns: (
    inhibitory <class 'hdmf.common.table.VectorData'>,
    depth <class 'hdmf.common.table.VectorData'>,
    layer <class 'hdmf.common.table.VectorData'>,
    neuron_channel <class 'hdmf.common.table.VectorData'>,
    spike_times_index <class 'hdmf.common.table.VectorIndex'>,
    spike_times <class 'hdmf.common.table.VectorData'>,
    obs_intervals_index <class 'hdmf.common.table.VectorIndex'>,
    obs_intervals <class 'hdmf.common.table.VectorData'>
  )
  description: Autogenerated by NWBFile
  id: id <class 'hdmf.common.table.ElementIdentifiers'>
  waveform_unit: volts

Trials:     start_time  stop_time  direct_delivery  stim_is_random  optogenetic  \
id                                                                        
88     399

In [3]:
from dandi.dandiapi import DandiAPIClient
from pynwb import NWBHDF5IO


with NWBHDF5IO(str(r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb"), "r") as io:
    nwb = io.read()

    # Find video references
    for name, obj in nwb.acquisition.items():
        if hasattr(obj, 'external_file'):
            print(f"{name}: {obj.external_file[:]}")
            print(f"  rate: {obj.rate}")
            print(f"  starting_time: {obj.starting_time}")

    # Find processed data (Kilosort results are here as units)
    if nwb.units:
        units_df = nwb.units.to_dataframe()
        print(f"\n{len(units_df)} sorted units (from Kilosort+Phy)")
        print(f"Columns: {units_df.columns.tolist()}")

    # Processing modules often contain behavioral or LFP data
    for mod_name, mod in nwb.processing.items():
        print(f"\nProcessing module '{mod_name}':")
        for container_name in mod.data_interfaces:
            print(f"  {container_name}: {type(mod[container_name]).__name__}")

behavioral video: ['sub-KF134_ses-20180211T154918_behavior+ecephys+image/26871407-4821-4cd0-bde7-38e14b3c13ba_external_file_0.mkv']
  rate: 200.0
  starting_time: 0.0

14 sorted units (from Kilosort+Phy)
Columns: ['inhibitory', 'depth', 'layer', 'neuron_channel', 'spike_times', 'obs_intervals']

Processing module 'behavior':
  licks: BehavioralEvents
  processed_position_whisker_C0: BehavioralTimeSeries
  processed_position_whisker_C1: BehavioralTimeSeries
  processed_position_whisker_C2: BehavioralTimeSeries
  processed_position_whisker_C3: BehavioralTimeSeries
  raw_position_whisker_C0: PoseEstimation
  raw_position_whisker_C1: PoseEstimation
  raw_position_whisker_C2: PoseEstimation
  raw_position_whisker_C3: PoseEstimation
  rewards: BehavioralEvents
  contacts_by_whisker_C0: TimeIntervals
  contacts_by_whisker_C1: TimeIntervals
  contacts_by_whisker_C2: TimeIntervals
  contacts_by_whisker_C3: TimeIntervals


In [3]:
from dandi.dandiapi import DandiAPIClient
from pathlib import Path

client = DandiAPIClient()
dandiset = client.get_dandiset("000231", "0.220904.1554")

for asset in dandiset.get_assets():
    if "KF134" in asset.path and "mkv" in asset.path:
        print(asset.path)
        filepath = Path(asset.path)
        filepath.parent.mkdir(parents=True, exist_ok=True)
        asset.download(filepath)
        break

sub-KF134/sub-KF134_ses-20180206T174211_behavior+image/b76384c3-94b4-479b-8169-9e8a7a833f6a_external_file_0.mkv


In [1]:
import pynwb

with pynwb.NWBHDF5IO(r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb", "r") as io:
    nwb = io.read()
    
    video = nwb.acquisition["behavioral video"]
    print(video.external_file[:])  # This tells you the filename
    print(video.format)            # Should be "external"

['sub-KF134_ses-20180211T154918_behavior+ecephys+image/26871407-4821-4cd0-bde7-38e14b3c13ba_external_file_0.mkv']
external


In [2]:
import pynwb

with pynwb.NWBHDF5IO(r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb", "r") as io:
    nwb = io.read()
    
    # Check for video/image data
    print(nwb.acquisition)
    
    # Look for ImageSeries
    for name, obj in nwb.acquisition.items():
        print(f"{name}: {type(obj).__name__}, shape: {getattr(obj, 'data', None).shape if hasattr(obj, 'data') else 'N/A'}")

{'behavioral video': behavioral video pynwb.image.ImageSeries at 0x2111836426160
Fields:
  comments: All times in the NWB file are defined with respect to this video, which starts at 0 and proceeds at exactly 200 fps
  conversion: 1.0
  data: <HDF5 dataset "data": shape (0, 0, 0), type "|u1">
  description: External file: behavioral video of whiskers
  device: high-speed camera pynwb.device.Device at 0x2111832668256
Fields:
  description: DR1-D1312IE-100-G2-8
  manufacturer: Photonfocus

  dimension: <HDF5 dataset "dimension": shape (2,), type "<i8">
  external_file: <StrDataset for HDF5 dataset "external_file": shape (1,), type "|O">
  format: external
  offset: 0.0
  rate: 200.0
  resolution: -1.0
  starting_frame: [0]
  starting_time: 0.0
  starting_time_unit: seconds
  unit: unknown
, 'extracellular array recording': extracellular array recording pynwb.ecephys.ElectricalSeries at 0x2111839910608
Fields:
  comments: broken channels excluded. original sampling was at 30 kHz but times

In [5]:
import h5py

f = h5py.File(r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb", "r")

# Navigate to the external_file dataset directly
ext = f['acquisition']['behavioral video']['external_file']
print(ext)
print(ext.shape)
print(ext[()])

<HDF5 dataset "external_file": shape (1,), type "|O">
(1,)
[b'sub-KF134_ses-20180211T154918_behavior+ecephys+image/26871407-4821-4cd0-bde7-38e14b3c13ba_external_file_0.mkv']


In [ ]:
import vocalpy as voc
from ethograph.features.energy import meansquared

sound = voc.Sound.read("path/to/audio.wav")
data = sound.data[0]  # first channel, 1-D
sr = sound.samplerate

env_time, envelope = meansquared(data, sr)

In [4]:
eid = eids[0]
video_path = one.load_dataset(eid, 'raw_video_data/_iblrig_leftCamera.raw.mp4')
print(video_path)

(S3) C:\Users\aksel\Downloads\ONE\openalyx.internationalbrainlab.org\churchlandlab_ucla\Subjects\FD_28\2023-12-01\001\raw_video_data\_iblrig_leftCamera.raw.mp4: 100%|██████████| 8.10G/8.10G [12:04<00:00, 11.2MB/s]

C:\Users\aksel\Downloads\ONE\openalyx.internationalbrainlab.org\churchlandlab_ucla\Subjects\FD_28\2023-12-01\001\raw_video_data\_iblrig_leftCamera.raw.mp4


In [ ]:
from dandi.dandiapi import DandiAPIClient
from pathlib import Path

dest = Path(r"C:\Users\aksel\Documents\Code\EthoGraph\data")

with DandiAPIClient() as client:
    ds = client.get_dandiset("000231")
    for asset in ds.get_assets():
        if "mkv" in asset.path:
            print(f"Downloading: {asset.path} ({asset.size / 1e6:.1f} MB)")
            asset.download(dest / Path(asset.path).name)
            print("Done")